In [4]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import from_unixtime, col,to_date, date_format


In [5]:
spark = SparkSession.builder.appName('movie').getOrCreate()


25/03/12 14:40:51 WARN Utils: Your hostname, ali-ThinkBook-14-G2-ITL resolves to a loopback address: 127.0.1.1; using 172.16.8.114 instead (on interface wlp0s20f3)
25/03/12 14:40:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/12 14:40:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df_data = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.data", 
                    sep="\t", 
                    schema="userId INT, movieId INT, rating FLOAT, timestamp LONG")

In [7]:
df_data.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



In [8]:
df_genre = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.genre", 
                           sep="|", 
                           schema="genre STRING, id INT")

df_info = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.info", 
                          sep=" ", 
                          schema="value INT, description STRING")

df_item = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.item", 
                          sep="|", 
                          schema="movieId INT, movieTitle STRING, releaseDate STRING, videoReleaseDate STRING, imdbUrl STRING, unknown INT, action INT, adventure INT, animation INT, childrens INT, comedy INT, crime INT, documentary INT, drama INT, fantasy INT, filmNoir INT, horror INT, musical INT, mystery INT, romance INT, sciFi INT, thriller INT, war INT, western INT")

df_occupation = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.occupation", 
                                 schema="occupation STRING")

df_user = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.user", 
                          sep="|", 
                          schema="userId INT, age INT, gender STRING, occupation STRING, zipCode STRING")

In [9]:
df_genre.show()

+-----------+---+
|      genre| id|
+-----------+---+
|    unknown|  0|
|     Action|  1|
|  Adventure|  2|
|  Animation|  3|
| Children's|  4|
|     Comedy|  5|
|      Crime|  6|
|Documentary|  7|
|      Drama|  8|
|    Fantasy|  9|
|  Film-Noir| 10|
|     Horror| 11|
|    Musical| 12|
|    Mystery| 13|
|    Romance| 14|
|     Sci-Fi| 15|
|   Thriller| 16|
|        War| 17|
|    Western| 18|
+-----------+---+



In [10]:
df_item.show(truncate=False)

+-------+----------------------------------------------------+-----------+----------------+-------------------------------------------------------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|movieId|movieTitle                                          |releaseDate|videoReleaseDate|imdbUrl                                                            |unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmNoir|horror|musical|mystery|romance|sciFi|thriller|war|western|
+-------+----------------------------------------------------+-----------+----------------+-------------------------------------------------------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|1      |Toy Story (1995)                                    |01-Ja

In [11]:
from pyspark.sql.functions import from_unixtime, col

df=df_data.join(df_user,'userId',how='inner').join(df_item,'movieId','inner').withColumn('datetime',from_unixtime(col("timestamp"))).drop('timestamp').withColumn('time',date_format(col("datetime"),"HH:mm:ss")).withColumn('date',to_date(col("datetime"))).drop('datetime')

In [12]:
df.show()

25/03/12 14:41:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------+------+---+------+-------------+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+--------+----------+
|movieId|userId|rating|age|gender|   occupation|zipCode|          movieTitle|releaseDate|videoReleaseDate|             imdbUrl|unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmNoir|horror|musical|mystery|romance|sciFi|thriller|war|western|    time|      date|
+-------+------+------+---+------+-------------+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+--------+----------+
|    242|   196|   3.0| 49|     M|       writer|  55105|        Kolya (1996)|24-Jan-1997|            NU

In [13]:
df_data=df_data.drop('timestamp')

In [14]:
df_data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



In [15]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [16]:
df_data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



In [17]:
df_data_tarin,df_data_test=df_data.randomSplit([0.8,0.2])

In [18]:
df_data_tarin.count()

79899

In [19]:
df_data_test.count()

20101

In [20]:
df_data.count()

100000

In [21]:
als=ALS(maxIter=10,rank=4,regParam=0.1,userCol='userId',itemCol='movieId',ratingCol='rating')

In [22]:
model=als.fit(df_data_tarin)

In [23]:
predictions=model.transform(df_data_test)

In [24]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      6|   5.0| 3.5571942|
|     1|     14|   5.0| 3.8853989|
|     1|     22|   4.0| 3.8085682|
|     1|     26|   3.0| 3.3011384|
|     1|     27|   2.0| 2.7762504|
|     1|     32|   5.0| 3.9521503|
|     1|     36|   2.0|  2.053674|
|     1|     46|   4.0| 3.6248262|
|     1|     47|   4.0| 3.9471827|
|     1|     55|   5.0| 3.8110852|
|     1|     64|   5.0|   4.32893|
|     1|     68|   4.0| 3.2708783|
|     1|     93|   5.0| 3.7443886|
|     1|     95|   4.0| 3.5746346|
|     1|    101|   2.0| 3.1710448|
|     1|    109|   5.0| 3.7101567|
|     1|    110|   1.0| 2.0539129|
|     1|    112|   1.0| 1.4567938|
|     1|    124|   5.0| 4.2176385|
|     1|    132|   4.0|  4.218403|
+------+-------+------+----------+
only showing top 20 rows



In [30]:
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [31]:
predictions=predictions.filter(col('prediction')!='nan')

In [32]:
rmse=evaluator.evaluate(predictions)

In [33]:
print(rmse)

0.9214316957753154


In [34]:
import datetime

In [41]:
print(datetime.datetime.now())

2025-03-12 15:02:16.738661


In [40]:
print(datetime.date.today())

2025-03-12


In [54]:
paramgrid=ParamGridBuilder().addGrid(als.regParam,[0.3,0.01,0.15]).addGrid(als.rank,range(2,8)).build()

In [55]:
crossval=CrossValidator(estimator=als,estimatorParamMaps=paramgrid,evaluator=evaluator,numFolds=5)

In [56]:
crossmodel=crossval.fit(df_data_tarin)

In [57]:
best_model=crossmodel.bestModel

In [58]:
predictions=best_model.transform(df_data_test)

In [59]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      6|   5.0|  3.514723|
|     1|     14|   5.0| 3.7462046|
|     1|     22|   4.0| 3.8386703|
|     1|     26|   3.0| 3.2385054|
|     1|     27|   2.0| 2.8747475|
|     1|     32|   5.0| 3.6679192|
|     1|     36|   2.0| 2.1226118|
|     1|     46|   4.0|  3.575832|
|     1|     47|   4.0| 3.5412688|
|     1|     55|   5.0|  3.508664|
|     1|     64|   5.0|  4.153926|
|     1|     68|   4.0| 3.2577043|
|     1|     93|   5.0|  3.482214|
|     1|     95|   4.0|  3.569971|
|     1|    101|   2.0| 3.1833694|
|     1|    109|   5.0| 3.2610004|
|     1|    110|   1.0|  2.381308|
|     1|    112|   1.0| 2.0861187|
|     1|    124|   5.0| 3.8501463|
|     1|    132|   4.0| 3.8335185|
+------+-------+------+----------+
only showing top 20 rows



In [60]:
rmse=evaluator.evaluate(predictions.filter(col('prediction')!='nan'))

In [61]:
print(rmse)

0.9832154482043312


In [53]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      6|   5.0| 3.5571942|
|     1|     14|   5.0| 3.8853989|
|     1|     22|   4.0| 3.8085682|
|     1|     26|   3.0| 3.3011384|
|     1|     27|   2.0| 2.7762504|
|     1|     32|   5.0| 3.9521503|
|     1|     36|   2.0|  2.053674|
|     1|     46|   4.0| 3.6248262|
|     1|     47|   4.0| 3.9471827|
|     1|     55|   5.0| 3.8110852|
|     1|     64|   5.0|   4.32893|
|     1|     68|   4.0| 3.2708783|
|     1|     93|   5.0| 3.7443886|
|     1|     95|   4.0| 3.5746346|
|     1|    101|   2.0| 3.1710448|
|     1|    109|   5.0| 3.7101567|
|     1|    110|   1.0| 2.0539129|
|     1|    112|   1.0| 1.4567938|
|     1|    124|   5.0| 4.2176385|
|     1|    132|   4.0|  4.218403|
+------+-------+------+----------+
only showing top 20 rows

